In [14]:
import ee
from geemap import geemap
import pandas as pd
import ee

In [12]:
ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')

geemap.plotlymap.Map

In [5]:
def mask_s2_clouds(image):
  # Quality assessment with resolution in meters
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# tci_bands = ['TCI_R', 'TCI_G', 'TCI_B']
# tci_max = 0.02550000138580799
rgb_max = 3000
rgb_bands = ['B4', 'B3', 'B2']
visualization = {
    'min': 0.0,
    'max': rgb_max,
    'bands': rgb_bands,
}

m = geemap.Map()
m.set_center(51.5, 0.1, 12)
m.addLayer(dataset.mean(), visualization, 'RGB')